In [1]:
from huggingface_hub import login
from datasets import load_dataset
import pandas as pd
from mlx_lm import load, generate, convert, lora

In [2]:
login()

In [ ]:
#model_path = 'meta-llama/Llama-3.2-1B'
#convert(model_path, q_bits=8, upload_repo='jbeiroa/mlx_llama-3.2-1b-q8')

In [3]:
model, tokenizer = load('jbeiroa/mlx_llama-3.2-1b-q8')

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
generate(model, tokenizer,
              prompt='Who was Albert Einstein?', verbose=True)

Prompt: Who was Albert Einstein?
Einstein was a German-born theoretical physicist who developed the general theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect". He is best known for his mass–energy equivalence formula E = mc2 (which has been dubbed "the world's most famous equation"), first proposed in 1905. His work is also known for its influence on the philosophy of science. He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect". He is best known for his mass–energy equivalence formula E = mc2 (which has been dubbed "the world's most famous equation"), first proposed in 1905. His work is also known for its influence on the philosophy of science.
Prompt: 6 tokens, 131.561 tokens-per-s

'Einstein was a German-born theoretical physicist who developed the general theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect". He is best known for his mass–energy equivalence formula E = mc2 (which has been dubbed "the world\'s most famous equation"), first proposed in 1905. His work is also known for its influence on the philosophy of science. He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect". He is best known for his mass–energy equivalence formula E = mc2 (which has been dubbed "the world\'s most famous equation"), first proposed in 1905. His work is also known for its influence on the philosophy of science.'

In [5]:
dataset = load_dataset('jbeiroa/resume_entities_ner_summaries')

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'annotation', 'highlight', 'length', 'summary'],
        num_rows: 154
    })
    test: Dataset({
        features: ['content', 'annotation', 'highlight', 'length', 'summary'],
        num_rows: 66
    })
})

In [20]:
train = dataset['train'].to_pandas()
_ = dataset['test'].to_pandas()
test = _.sample(frac=0.5)
msk = test.index
valid = _.loc[~_.index.isin(msk)].copy()

In [8]:
train.sample(5)

,content,annotation,highlight,length,summary
117,Ayushi Srivastava\nSenior Analyst - Cisco\n\nN...,"[{'label': ['Email Address'], 'points': [{'end...",{'College Name': 'Sumermal Jain Public School ...,2110,*Name*: Ayushi Srivastava.\n*Skills*: CSS (Les...
45,"Harini Komaravelli\nTest Analyst at Oracle, Hy...","[{'label': ['Companies worked at'], 'points': ...",{'College Name': 'Osmania University Osmania U...,5203,*Name*: Harini Komaravelli.\n*Skills*: Functio...
153,Avin Sharma\nSenior Associate Consultant - Inf...,"[{'label': ['Skills'], 'points': [{'end': 2478...",{'College Name': 'Great Lakes Institute of Man...,2479,*Name*: Avin Sharma.\n*Skills*: Bid management...
92,"Vikas Singh\nChandigarh, Chandigarh - Email me...","[{'label': ['Skills'], 'points': [{'end': 5749...",{'College Name': 'GLA Institute of Technology ...,5750,*Name*: Vikas Singh.\n*Skills*: SECURITY (5 ye...
3,Roshan Sinha\nApplication Developer - SAP ABAP...,"[{'label': ['Skills'], 'points': [{'end': 3263...","{'College Name': 'Anna University Chennai', 'C...",3264,*Name*: Roshan Sinha.\n*Skills*: OOPS-ABAP\nSA...


In [9]:
test.index

Index([27,  5, 22, 29, 16,  2,  1, 39,  4, 37, 26, 47, 30, 20, 59, 36, 43, 56,
       61, 17,  3, 10, 31, 12, 25, 45, 60, 55, 64, 44, 41, 42, 14],
      dtype='int64')

In [10]:
valid.index

Index([ 0,  6,  7,  8,  9, 11, 13, 15, 18, 19, 21, 23, 24, 28, 32, 33, 34, 35,
       38, 40, 46, 48, 49, 50, 51, 52, 53, 54, 57, 58, 62, 63, 65],
      dtype='int64')

In [11]:
train.summary[0]

'*Name*: Mohini Gupta.\n*Skills*: active directory, iis, sccm, dhcp, sql, wsus, dns\n\nADDITIONAL INFORMATION\n\nComputer Skills\n● MS Office Tools: MS Excel, MS Word, MS Power Point.\n● Hands on experience on all versions of Windows.\n● Sound knowledge of internet and networking.\n● Coding Languages: C, C++, Java..\n*College Name*: KIIT college of Engg..\n*Degree*: B.tech.\n*Companies worked at*: Microsoft\nMicrosoft.\n*Designation*: Server Support Engineer\nServer Support Engineer.\n'

In [21]:
build_instruction = lambda content, summary: f"""<|begin_of_text|><|start_header_id|>system><|end_header_id|>
You are a helpful human resources assistant, specialized in resume analysis and data extraction from job candidate applicants. 
When asked to summarize a resume, you should structure your output in this format format:
*Name*: [Candidate name]
*Skills*: [Candidate skills]
*Degree*: [Candidate degrees]
*Companies worked at*: [Candidate job experience]
*Designation*: [Candidate job roles]<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task: summarize the following resume using the specified structure:   
{content}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
{summary}"""

In [22]:
train['text'] = train.apply(lambda x: build_instruction(x.content, x.summary), axis=1)
test['text'] = test.apply(lambda x: build_instruction(x.content, x.summary), axis=1)
valid['text'] = valid.apply(lambda x: build_instruction(x.content, x.summary), axis=1)

In [23]:
train['tokenized'] = train.apply(lambda x: tokenizer.encode(x.text), axis=1)
test['tokenized'] = train.apply(lambda x: tokenizer.encode(x.text), axis=1)
valid['tokenized'] = train.apply(lambda x: tokenizer.encode(x.text), axis=1)

In [24]:
train.shape, valid.shape, test.shape

((154, 7), (33, 7), (33, 7))

In [25]:
train = train[train.tokenized.map(len) <= 2048]
test = test[test.tokenized.map(len) <= 2048]
valid = valid[valid.tokenized.map(len) <= 2048]

In [26]:
train.shape, valid.shape, test.shape

((142, 7), (31, 7), (31, 7))

In [27]:
import json

# Path to save the JSONL file
output_file = "train.jsonl"

# Writing to JSONL
with open(output_file, 'w', encoding='utf-8') as f:
    for _, row in train.iterrows():  # Iterate over each row
        json_line = {'text': row['text']}  # Wrap each text in a dictionary
        f.write(json.dumps(json_line) + '\n')  # Convert dict to JSON string and write

In [28]:
# Path to save the JSONL file
output_file = "test.jsonl"

# Writing to JSONL
with open(output_file, 'w', encoding='utf-8') as f:
    for _, row in test.iterrows():  # Iterate over each row
        json_line = {'text': row['text']}  # Wrap each text in a dictionary
        f.write(json.dumps(json_line) + '\n')  # Convert dict to JSON string and write

In [29]:
# Path to save the JSONL file
output_file = "valid.jsonl"

# Writing to JSONL
with open(output_file, 'w', encoding='utf-8') as f:
    for _, row in valid.iterrows():  # Iterate over each row
        json_line = {'text': row['text']}  # Wrap each text in a dictionary
        f.write(json.dumps(json_line) + '\n')  # Convert dict to JSON string and write

In [30]:
!mlx_lm.lora --model jbeiroa/mlx_llama-3.2-1b-q8 --train --fine-tune-type lora --num-layers 16 --iters 600 --learning-rate 1e-6 --batch-size 2 --steps-per-report 60 --steps-per-eval 60 --test --grad-checkpoint

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pretrained model
Fetching 6 files: 100%|████████████████████████| 6/6 [00:00<00:00, 19166.66it/s]
Loading datasets
Training
Trainable parameters: 0.069% (0.852M/1235.814M)
Starting training..., iters: 600
Iter 1: Val loss 2.718, Val took 31.294s
Iter 60: Val loss 2.647, Val took 39.763s
Iter 60: Train loss 2.913, Learning Rate 1.000e-06, It/sec 16.515, Tokens/sec 30842.330, Trained Tokens 112051, Peak mem 14.342 GB
Iter 100: Saved adapter weights to adapters/adapters.safetensors and adapters/0000100_adapters.safetensors.
Iter 120: Val loss 2.595, Val took 40.880s
Iter 120: Train loss 2.794, Learning Rate 1.000e-06, It/sec 10.257, Tokens/sec 19115.667, Trained Tokens 223866, Peak mem 14.398 GB
Iter 180: Val loss 2.553, Val took 45.680s
Iter 180: Train loss 2.741, Learning Rate 1.000e-06, It/sec 3.915, Tokens/sec 7389.507, Trained Tokens 337121, Peak mem 14.398 GB
Iter 200: Saved adapter weights to adapters/adapters.safetensors and adapters/0000200_adapters.safetensors.
Iter 240:

In [31]:
!mlx_lm.fuse --model jbeiroa/mlx_llama-3.2-1b-q8 --upload-repo jbeiroa/mlx_lora_llama-3.2-1b-q8

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pretrained model
Fetching 6 files: 100%|████████████████████████| 6/6 [00:00<00:00, 51781.53it/s]
Start hashing 7 files.
Finished hashing 7 files.
/Users/juanbeiroa/Library/Caches/pypoetry/virtualenvs/thereisnohr-HzDUGj03-py3.12/lib/python3.12/site-packages/huggingface_hub/utils/_experimental.py:58: UserWarning: 'plan_multi_commits' is experimental and might be subject to breaking changes in the future. You can disable this warning by setting `HF_HUB_DISABLE_EXPERIMENTAL_WARNING=1` as environment variable.
  warnings.warn(
/Users/juanbeiroa/Library/Caches/pypoetry/virtualenvs/thereisnohr-HzDUGj03-py3.12/lib/python3.12/site-packages/huggingface_hub/utils/_experimental.py:58: UserWarning: 'HfApi.create_commits_on_pr' is experimental and might be subject to breaking changes in the future. You can disable this warning by setting `HF_HUB_DISABLE_EXPERIMENTAL_WARNING=1` as environment variable.
  warnings.warn(
/Users/juanbeiroa/Library/Caches/pypoetry/virtualenvs/thereisnohr-HzDUGj0